In [1]:
pip install biopandas lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 KB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from biopandas.pdb import PandasPdb

import time

from sklearn.model_selection import KFold
from scipy.stats import spearmanr
from lightgbm import LGBMRegressor


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Novozymes-Enzyme-Stability-Prediction/train.csv'
file_key_2 = 'Novozymes-Enzyme-Stability-Prediction/test.csv'
file_key_3 = 'Novozymes-Enzyme-Stability-Prediction/wildtype_structure_prediction_af2.pdb'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
# pdb_df =  PandasPdb().read_pdb(file_content_stream_3)
# pdb_df.df.keys()

In [ ]:
gt = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

def position(a):
    if a != gt:
        value = [i for i in range(len(a)) if a[i] != gt[i]][0]
    else:
        value = 0
    return value

In [10]:
test['position_from_start'] = test.protein_sequence.apply(position)
test['position_from_end']   = len(gt) - test['position_from_start']

In [11]:
test.head()

,seq_id,protein_sequence,pH,data_source,position_from_start,position_from_end
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,16,205
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,16,205
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,16,205
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17,204
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,17,204


In [12]:
len(gt)

221

# Reading pdb file

In [1]:
pip install biopandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 KB 41.1 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
from biopandas.pdb import PandasPdb

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

pdb_df =  PandasPdb().read_pdb('AF-P61989-F1-model_v3.pdb')
pdb_df.df.keys()

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])

In [29]:
atom_df = pdb_df.df['ATOM']
hetatm_df = pdb_df.df['HETATM']
anisou_df = pdb_df.df['ANISOU']
others_df = pdb_df.df['OTHERS']
print(f"ATOM dataset is of shape: {atom_df.shape}") 
print(f"HETATM dataset is of shape: {hetatm_df.shape}")
print(f"ANISOU dataset is of shape: {anisou_df.shape}")
print(f"OTHERS dataset is of shape: {others_df.shape}")
display(atom_df.head(10))
display(others_df.head())

ATOM dataset is of shape: (2742, 21)
HETATM dataset is of shape: (0, 21)
ANISOU dataset is of shape: (0, 21)
OTHERS dataset is of shape: (79, 3)


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,MET,,A,1,,...,35.892,10.745,-0.925,1.0,33.09,,,N,NaN,76
1,ATOM,2,,CA,,MET,,A,1,,...,35.670,11.965,-1.730,1.0,33.09,,,C,NaN,77
2,ATOM,3,,C,,MET,,A,1,,...,34.246,12.427,-1.479,1.0,33.09,,,C,NaN,78
3,ATOM,4,,CB,,MET,,A,1,,...,35.894,11.695,-3.226,1.0,33.09,,,C,NaN,79
4,ATOM,5,,O,,MET,,A,1,,...,33.338,11.627,-1.661,1.0,33.09,,,O,NaN,80
5,ATOM,6,,CG,,MET,,A,1,,...,37.376,11.490,-3.564,1.0,33.09,,,C,NaN,81
6,ATOM,7,,SD,,MET,,A,1,,...,37.637,10.984,-5.279,1.0,33.09,,,S,NaN,82
7,ATOM,8,,CE,,MET,,A,1,,...,39.436,10.738,-5.287,1.0,33.09,,,C,NaN,83
8,ATOM,9,,N,,SER,,A,2,,...,34.052,13.645,-0.974,1.0,38.24,,,N,NaN,84
9,ATOM,10,,CA,,SER,,A,2,,...,32.722,14.244,-0.813,1.0,38.24,,,C,NaN,85


,record_name,entry,line_idx
0,HEADER,01...,0
1,TITLE,ALPHAFOLD MONOMER V2.0 PREDICTION FOR COBA...,1
2,TITLE,2 C(1)-METHYLTRANSFERASE (P61989),2
3,COMPND,MOL_ID: 1;,3
4,COMPND,2 MOLECULE: COBALT-PRECORRIN-5B C(1)-METHYL...,4


In [30]:
atom_df['residue_number'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [24]:
x = 'AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR'
len(x)

341

In [31]:
residue_agg = pd.DataFrame(atom_df.groupby(['residue_name', 'residue_number'])[['x_coord', 'y_coord', 'z_coord']].mean())
residue_agg['residue_name'] = residue_agg.index.get_level_values(0)
residue_agg['residue_number'] = residue_agg.index.get_level_values(1)
residue_agg = residue_agg.reset_index(drop = True)
residue_agg = residue_agg[['residue_name', 'residue_number', 'x_coord', 'y_coord', 'z_coord']].sort_values(by = 'residue_number').reset_index(drop = True)
display(residue_agg.head(10))

,residue_name,residue_number,x_coord,y_coord,z_coord
0,MET,1,36.186125,11.458875,-2.893875
1,SER,2,33.042833,14.813333,-1.334167
2,HIS,3,28.489200,13.546300,-4.191000
3,PRO,4,30.088429,16.780571,-6.206714
4,TYR,5,26.698833,17.820333,-7.485333
5,PRO,6,25.555571,18.196000,-1.787143
6,PRO,7,22.678714,15.561571,-0.603571
7,PRO,8,19.248714,17.658143,-1.065286
8,ARG,9,20.854727,18.509273,4.341909
9,ASP,10,15.178625,18.689625,5.422125


In [32]:
residue_agg.describe()

,residue_number,x_coord,y_coord,z_coord
count,366.000000,366.000000,366.000000,366.000000
mean,183.500000,1.745672,3.549513,1.838712
std,105.799338,15.297623,7.939275,17.426021
min,1.000000,-25.699444,-13.204286,-33.497500
25%,92.250000,-11.150522,-2.566594,-10.503804
50%,183.500000,-0.773062,3.619903,0.603125
75%,274.750000,14.710194,9.907000,14.262469
max,366.000000,36.186125,23.051444,42.724667


In [33]:
scaler = MinMaxScaler()

residue_agg[['x_coord', 'y_coord', 'z_coord']] = pd.DataFrame(scaler.fit_transform(residue_agg[['x_coord', 'y_coord', 'z_coord']]), columns = ['x_coord', 'y_coord', 'z_coord'])
residue_agg.head(10)

,residue_name,residue_number,x_coord,y_coord,z_coord
0,MET,1,1.000000,0.680256,0.401506
1,SER,2,0.949208,0.772778,0.421968
2,HIS,3,0.875626,0.737831,0.384488
3,PRO,4,0.901468,0.827038,0.358043
4,TYR,5,0.846696,0.855716,0.341268
5,PRO,6,0.828222,0.866078,0.416025
6,PRO,7,0.781736,0.793415,0.431553
7,PRO,8,0.726311,0.851243,0.425496
8,ARG,9,0.752262,0.874719,0.496436
9,ASP,10,0.660543,0.879693,0.510608


In [41]:
nbrs = NearestNeighbors(n_neighbors = 6).fit(residue_agg[['x_coord', 'y_coord', 'z_coord']])

distances, indices = nbrs.kneighbors(residue_agg[['x_coord', 'y_coord', 'z_coord']])

In [45]:
names = ['neighbor_' + str(i) for i in range(1, 5)]
names

['neighbor_1', 'neighbor_2', 'neighbor_3', 'neighbor_4']

In [42]:
pd.DataFrame(indices, )

,0,1,2,3,4,5
0,0,1,2,55,40,3
1,1,2,3,0,5,4
2,2,1,3,6,53,4
3,3,4,1,2,5,53
4,4,3,5,53,6,2
...,...,...,...,...,...,...
361,361,362,360,363,359,364
362,362,363,361,220,360,219
363,363,362,353,360,220,219
364,364,365,221,222,219,220


In [53]:
indices[:,1:] + 1

array([[  2,   3,  56,  41,   4],
       [  3,   4,   1,   6,   5],
       [  2,   4,   7,  54,   5],
       ...,
       [363, 354, 361, 221, 220],
       [366, 222, 223, 220, 221],
       [219, 220, 365, 222, 353]])

In [54]:
def amino_acid_neighbors(pdb_file, k):
    
    """
    This function appends the k-neighbors aminoacids 
    for a given pbd file. 
    
    pdb_file: the file 
    k: the number of neighbors
    """
    
    ## Reading the file 
    pdb_df =  PandasPdb().read_pdb(pdb_file)

    ## Extracting atom 
    atom_df = pdb_df.df['ATOM']
    
    ## Aggregating based on residue
    residue_agg = pd.DataFrame(atom_df.groupby(['residue_name', 'residue_number'])[['x_coord', 'y_coord', 'z_coord']].mean())
    residue_agg['residue_name'] = residue_agg.index.get_level_values(0)
    residue_agg['residue_number'] = residue_agg.index.get_level_values(1)
    residue_agg = residue_agg.reset_index(drop = True)
    residue_agg = residue_agg[['residue_name', 'residue_number', 'x_coord', 'y_coord', 'z_coord']].sort_values(by = 'residue_number').reset_index(drop = True)
    
    ## Scaling x-y-z coordinates
    scaler = MinMaxScaler()
    scaled_data = pd.DataFrame(scaler.fit_transform(residue_agg[['x_coord', 'y_coord', 'z_coord']]), columns = ['x_coord', 'y_coord', 'z_coord'])
    
    ## Finding the k-neighbors
    nbrs = NearestNeighbors(n_neighbors = (k + 1)).fit(scaled_data)
    distances, indices = nbrs.kneighbors(scaled_data)
    
    neighbors_names = ['neighbor_' + str(i) for i in range(1, (k + 1))]
    neighbors = pd.DataFrame(indices[:, 1:] + 1, columns = neighbors_names)
    
    ## Appending neighbors
    residue = pd.concat([residue_agg, neighbors], axis = 1) 
    
    return residue

In [55]:
out = amino_acid_neighbors('AF-P61989-F1-model_v3.pdb', 5)

In [56]:
out

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,36.186125,11.458875,-2.893875,2,3,56,41,4
1,SER,2,33.042833,14.813333,-1.334167,3,4,1,6,5
2,HIS,3,28.489200,13.546300,-4.191000,2,4,7,54,5
3,PRO,4,30.088429,16.780571,-6.206714,5,2,3,6,54
4,TYR,5,26.698833,17.820333,-7.485333,4,6,54,7,3
...,...,...,...,...,...,...,...,...,...,...
361,GLU,362,-25.699444,-6.493222,14.445333,363,361,364,360,365
362,VAL,363,-21.418714,-6.795571,11.072857,364,362,221,361,220
363,TYR,364,-18.355750,-5.402917,13.719083,363,354,361,221,220
364,ARG,365,-21.241455,-11.982545,12.691000,366,222,223,220,221


# Reading More Files

In [1]:
pip install biopandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 KB 16.3 MB/s eta 0:00:0000:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [52]:
# import pandas as pd
# from biopandas.pdb import PandasPdb

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neighbors import NearestNeighbors
from Finding_Neighbors import amino_acid_neighbors

data_1 = amino_acid_neighbors('AF-P61989-F1-model_v3.pdb', 5)
data_2 = amino_acid_neighbors('AF-Q27237-F1-model_v3.pdb', 5)
data_3 = amino_acid_neighbors('AF-Q80WW9-F1-model_v3.pdb', 5)

In [53]:
data_1.head(60)

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,36.186125,11.458875,-2.893875,2,3,56,41,4
1,SER,2,33.042833,14.813333,-1.334167,3,4,1,6,5
2,HIS,3,28.489200,13.546300,-4.191000,2,4,7,54,5
3,PRO,4,30.088429,16.780571,-6.206714,5,2,3,6,54
4,TYR,5,26.698833,17.820333,-7.485333,4,6,54,7,3
5,PRO,6,25.555571,18.196000,-1.787143,5,7,4,8,9
6,PRO,7,22.678714,15.561571,-0.603571,8,6,16,15,9
7,PRO,8,19.248714,17.658143,-1.065286,9,7,16,6,10
8,ARG,9,20.854727,18.509273,4.341909,13,8,10,14,7
9,ASP,10,15.178625,18.689625,5.422125,12,13,9,14,8


In [35]:
data_1.describe()

,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,183.500000,1.745672,3.549513,1.838712,182.756831,183.398907,181.377049,182.663934,183.270492
std,105.799338,15.297623,7.939275,17.426021,106.788093,103.702126,105.797817,105.138704,104.433635
min,1.000000,-25.699444,-13.204286,-33.497500,2.000000,2.000000,1.000000,6.000000,3.000000
25%,92.250000,-11.150522,-2.566594,-10.503804,88.250000,94.000000,91.250000,93.000000,94.000000
50%,183.500000,-0.773062,3.619903,0.603125,184.500000,183.000000,185.500000,184.000000,188.000000
75%,274.750000,14.710194,9.907000,14.262469,277.250000,269.250000,272.750000,272.500000,270.750000
max,366.000000,36.186125,23.051444,42.724667,366.000000,366.000000,366.000000,366.000000,365.000000


In [6]:
data_2

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,7.717000,38.068625,-4.504875,2,3,4,5,6
1,MET,2,2.323125,36.900250,-4.430500,3,1,4,5,6
2,ILE,3,2.286250,35.703375,-9.193875,2,4,5,1,6
3,SER,4,-0.960667,32.534833,-9.351333,5,3,6,2,7
4,CYS,5,-1.911000,31.826167,-13.186000,4,6,7,3,2
...,...,...,...,...,...,...,...,...,...,...
515,LYS,516,-20.191667,20.542444,41.157778,515,517,518,514,513
516,SER,517,-24.726500,24.058667,41.943667,518,515,516,514,519
517,MET,518,-28.459250,22.838750,40.130500,517,519,520,515,516
518,PHE,519,-30.159091,18.305909,40.967000,520,518,517,516,515


In [7]:
data_3

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,-18.821375,8.724000,-11.853500,2,3,4,6,7
1,VAL,2,-14.262429,10.172143,-9.445857,3,1,6,7,4
2,GLY,3,-12.468500,11.493000,-12.471000,4,2,6,7,5
3,PRO,4,-9.993429,10.451571,-14.698714,3,5,7,6,2
4,TRP,5,-8.185214,14.816286,-15.899786,4,8,3,6,9
...,...,...,...,...,...,...,...,...,...,...
310,ALA,311,-29.119000,-31.552800,29.417800,310,312,309,313,314
311,GLN,312,-33.120556,-31.879222,27.013778,313,311,314,310,309
312,ALA,313,-33.971400,-34.447800,31.088800,314,312,315,311,310
313,SER,314,-33.772000,-38.049333,30.092167,313,315,312,311,310


In [48]:
import pandas as pd
from biopandas.pdb import PandasPdb

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

def amino_acid_neighbors(pdb_file, k):
    
    """
    This function appends the k-neighbors aminoacids 
    for a given pbd file. 
    
    pdb_file: the file 
    k: the number of neighbors
    """
    
    ## Reading the file 
    pdb_df =  PandasPdb().read_pdb(pdb_file)

    ## Extracting atom 
    atom_df = pdb_df.df['ATOM']
    
    ## Aggregating based on residue
    residue_agg = pd.DataFrame(atom_df.groupby(['residue_name', 'residue_number'])[['x_coord', 'y_coord', 'z_coord']].mean())
    residue_agg['residue_name'] = residue_agg.index.get_level_values(0)
    residue_agg['residue_number'] = residue_agg.index.get_level_values(1)
    residue_agg = residue_agg.reset_index(drop = True)
    residue_agg = residue_agg[['residue_name', 'residue_number', 'x_coord', 'y_coord', 'z_coord']].sort_values(by = 'residue_number').reset_index(drop = True)
    
    ## Scaling x-y-z coordinates
    scaler = MinMaxScaler()
    scaled_data = pd.DataFrame(scaler.fit_transform(residue_agg[['x_coord', 'y_coord', 'z_coord']]), columns = ['x_coord', 'y_coord', 'z_coord'])
    
    ## Finding the k-neighbors
    nbrs = NearestNeighbors(n_neighbors = (k + 1)).fit(scaled_data)
    distances, indices = nbrs.kneighbors(scaled_data)
    
    neighbors_names = ['neighbor_' + str(i) for i in range(1, (k + 1))]
    
    indices = indices[:, 1:] 
    residue_names = list()
    for i in range(0, indices.shape[0]):
        residue_names.append(residue_agg['residue_name'][indices[i]].values)
    
    neighbors = pd.DataFrame(residue_names, columns = neighbors_names)
    
    ## Appending neighbors
    residue = pd.concat([residue_agg, neighbors], axis = 1) 
    
    return residue

In [49]:
data_1 = amino_acid_neighbors('AF-P61989-F1-model_v3.pdb', 5)

In [50]:
data_1

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,36.186125,11.458875,-2.893875,SER,HIS,ARG,GLU,PRO
1,SER,2,33.042833,14.813333,-1.334167,HIS,PRO,MET,PRO,TYR
2,HIS,3,28.489200,13.546300,-4.191000,SER,PRO,PRO,PRO,TYR
3,PRO,4,30.088429,16.780571,-6.206714,TYR,SER,HIS,PRO,PRO
4,TYR,5,26.698833,17.820333,-7.485333,PRO,PRO,PRO,PRO,HIS
...,...,...,...,...,...,...,...,...,...,...
361,GLU,362,-25.699444,-6.493222,14.445333,VAL,ARG,TYR,ASP,ARG
362,VAL,363,-21.418714,-6.795571,11.072857,TYR,GLU,LEU,ARG,LEU
363,TYR,364,-18.355750,-5.402917,13.719083,VAL,CYS,ARG,LEU,LEU
364,ARG,365,-21.241455,-11.982545,12.691000,GLY,PRO,HIS,LEU,LEU


In [16]:
Y = data_1[1]
Y

,residue_name,residue_number,x_coord,y_coord,z_coord,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5
0,MET,1,36.186125,11.458875,-2.893875,2,3,56,41,4
1,SER,2,33.042833,14.813333,-1.334167,3,4,1,6,5
2,HIS,3,28.489200,13.546300,-4.191000,2,4,7,54,5
3,PRO,4,30.088429,16.780571,-6.206714,5,2,3,6,54
4,TYR,5,26.698833,17.820333,-7.485333,4,6,54,7,3
...,...,...,...,...,...,...,...,...,...,...
361,GLU,362,-25.699444,-6.493222,14.445333,363,361,364,360,365
362,VAL,363,-21.418714,-6.795571,11.072857,364,362,221,361,220
363,TYR,364,-18.355750,-5.402917,13.719083,363,354,361,221,220
364,ARG,365,-21.241455,-11.982545,12.691000,366,222,223,220,221


In [24]:
Y['residue_name'][X[2] + 1]

2     HIS
4     TYR
7     PRO
54    PHE
5     PRO
Name: residue_name, dtype: object

In [23]:
X = data_1[0]
X[2]

array([ 1,  3,  6, 53,  4])